# Volume 3 Project

This is where we will work on our volume 3 project. Remember to pull before working and to push after making any edits!

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import statsmodels.api as sm
import sklearn
from scipy import stats
from statsmodels.regression.linear_model import OLS
from sklearn import linear_model as lm
from sklearn import naive_bayes
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
github_url = "https://raw.githubusercontent.com/pjfort/pell_grants/main/data/raw/"

# Data Cleaning
Data on college atrributes comes from https://opportunityinsights.org/data/
Pell grant data comes from  https://www2.ed.gov/finaid/prof/resources/data/pell-institution.html

In [3]:
# Load in Opportunity Insight data
mobility_df = pd.read_csv(github_url + "mrc_table1.csv")
characteristics_df = pd.read_csv(github_url + "mrc_table10.csv")

# Merge dataframes
mobility_df = mobility_df.drop(columns=['name', 'state', 'czname']) 
df = mobility_df.merge(characteristics_df, on='super_opeid', how="outer")

In [5]:
# Fill in missing data

# Data coming from the Mobility table
# There are ~264 nans in here out of ~2466.
df["par_median"] = df["par_median"].fillna(df["par_median"].mean())
df["k_median"] = df["k_median"].fillna(df["k_median"].mean())
df["par_q1"] = df["par_q1"].fillna(df["par_q1"].mean())
df["par_top1pc"] = df["par_top1pc"].fillna(df["par_top1pc"].mean())
df["kq5_cond_parq1"] = df["kq5_cond_parq1"].fillna(df["kq5_cond_parq1"].mean())
df["ktop1pc_cond_parq1"] = df["ktop1pc_cond_parq1"].fillna(df["ktop1pc_cond_parq1"].mean())
df["mr_kq5_pq1"] = df["mr_kq5_pq1"].fillna(df["mr_kq5_pq1"].mean())
df["mr_ktop1_pq1"] = df["mr_ktop1_pq1"].fillna(df["mr_ktop1_pq1"].mean())
df["trend_parq1"] = df["trend_parq1"].fillna(df["trend_parq1"].mean())
df["trend_bottom40"] = df["trend_bottom40"].fillna(df["trend_bottom40"].mean())
df["count"] = df["count"].fillna(df["count"].mean())

# Data from College Characteristics table
df["county"] = df["county"].fillna("none")

# If a school has nan here, they probably actually spent money on instruction
df["exp_instr_pc_2000"] = df["exp_instr_pc_2000"].fillna(df["exp_instr_pc_2000"].mean())
df["exp_instr_pc_2013"] = df["exp_instr_pc_2013"].fillna(df["exp_instr_pc_2013"].mean())
#df["multi"] = df["multi"].fillna(df["multi"].mean())
#df["hbcu"] = df["hbcu"].fillna(df["hbcu"].mean())
#df["flagship"] = df["flagship"].fillna(df["flagship"].mean())

# I'm not sure what this is.
df["ipeds_enrollment_2013"] = df["ipeds_enrollment_2013"].fillna(df["ipeds_enrollment_2013"].mean())
df["ipeds_enrollment_2000"] = df["ipeds_enrollment_2000"].fillna(df["ipeds_enrollment_2000"].mean())

# The school probably does cost something instead of nan. Let's do the mean
df["sticker_price_2013"] = df["sticker_price_2013"].fillna(df["sticker_price_2013"].mean())
df["sticker_price_2000"] = df["sticker_price_2000"].fillna(df["sticker_price_2000"].mean())

# Not quite sure about this column. Going to set to mean since the mean seams to be around 50%
df["grad_rate_150_p_2013"] = df["grad_rate_150_p_2013"].fillna(df["grad_rate_150_p_2013"].mean())
df["grad_rate_150_p_2002"] = df["grad_rate_150_p_2002"].fillna(df["grad_rate_150_p_2002"].mean())

# Giving teachers the average salary of other teachers makes sense here. 
df["avgfacsal_2013"] = df["avgfacsal_2013"].fillna(df["avgfacsal_2013"].mean())
df["avgfacsal_2001"] = df["avgfacsal_2001"].fillna(df["avgfacsal_2001"].mean())

# I will include this column. Too much of the data is missing for me to feel comfortable setting it to the mean. If you want to use it, get rid of the zero values.
df["sat_avg_2013"] = df["sat_avg_2013"].fillna(0)
df["sat_avg_2001"] = df["sat_avg_2001"].fillna(0)

# Probably okay to set to mean. Also not much missing data
df["scorecard_netprice_2013"] = df["scorecard_netprice_2013"].fillna(df["scorecard_netprice_2013"].mean())

# Too much of the data is missing. I am dropping this column.
df = df.drop(columns=["scorecard_rej_rate_2013"])
#df["scorecard_rej_rate_2013"] = df["scorecard_rej_rate_2013"].fillna(df["scorecard_rej_rate_2013"].mean())

# Setting to the mean seems alright for this.
df["scorecard_median_earnings_2011"] = df["scorecard_median_earnings_2011"].fillna(df["scorecard_median_earnings_2011"].mean())

# I'm not quite sure what this entails and quite a bit of data is missing. I'm dropping it.
df = df.drop(columns=["endowment_pc_2000"])
#df["endowment_pc_2000"] = df["endowment_pc_2000"].fillna(df["endowment_pc_2000"].mean())

# I feel comfortable setting nans to the mean here.
df["exp_instr_2012"] = df["exp_instr_2012"].fillna(df["exp_instr_2012"].mean())
df["exp_instr_2000"] = df["exp_instr_2000"].fillna(df["exp_instr_2000"].mean())

# For the rest of these, there isn't a ton of data missing, so I'll drop the rows.
df["asian_or_pacific_share_fall_2000"] = df["asian_or_pacific_share_fall_2000"].fillna(df["asian_or_pacific_share_fall_2000"].mean())
df["black_share_fall_2000"] = df["black_share_fall_2000"].fillna(df["black_share_fall_2000"].mean())
df["hisp_share_fall_2000"] = df["hisp_share_fall_2000"].fillna(df["hisp_share_fall_2000"].mean())
df["alien_share_fall_2000"] = df["alien_share_fall_2000"].fillna(df["alien_share_fall_2000"].mean())
df["pct_arthuman_2000"] = df["pct_arthuman_2000"].fillna(df["pct_arthuman_2000"].mean())
df["pct_business_2000"] = df["pct_business_2000"].fillna(df["pct_business_2000"].mean())
df["pct_health_2000"] = df["pct_health_2000"].fillna(df["pct_health_2000"].mean())
df["pct_multidisci_2000"] = df["pct_multidisci_2000"].fillna(df["pct_multidisci_2000"].mean())
df["pct_publicsocial_2000"] = df["pct_publicsocial_2000"].fillna(df["pct_publicsocial_2000"].mean())
df["pct_stem_2000"] = df["pct_stem_2000"].fillna(df["pct_stem_2000"].mean())
df["pct_socialscience_2000"] = df["pct_socialscience_2000"].fillna(df["pct_socialscience_2000"].mean())
df["pct_tradepersonal_2000"] = df["pct_tradepersonal_2000"].fillna(df["pct_tradepersonal_2000"].mean())

# Drop the nan rows
df = df.dropna()

In [6]:
# Create crosswalk between one-to-many super_opeid and opeid
opeid_crosswalk = pd.read_csv(github_url + "mrc_table11.csv")
opeid_crosswalk = opeid_crosswalk.loc[(opeid_crosswalk['multi'] == 1) & (opeid_crosswalk['super_opeid'] != -1)]
super_opeid_lookup = dict()
for id, superid in zip(opeid_crosswalk['opeid'], opeid_crosswalk['super_opeid']):
        super_opeid_lookup.update({str(id) + '00':str(superid) + '00'})

# Format super_opeid
df['super_opeid'] = df['super_opeid'].astype('str') + '00'

# Create crosswalk between pellid and opeid
crosswalk_data = pd.read_excel(github_url + "pell-inst-03-04.xls")

pell_crosswalk = crosswalk_data[['OPEID', 'PELLID']]
pell_crosswalk = pell_crosswalk.dropna()
pell_crosswalk['OPEID'] = pell_crosswalk['OPEID'].astype(int)

In [7]:
# Helper function to merge in pell grant data
def add_pell_grants(df, year, ftype, vars, id, add_opeid=False, add_zeros=False):
    # Load in data
    pell_df = pd.read_excel(github_url + "pell-inst-"+ year + ftype)
    if add_opeid:
        pell_df = pell_df.merge(pell_crosswalk, on='PELLID', how='left')
    
    # Keep relevant variables
    pell_df = pell_df[[id, vars[0], vars[1]]]
    pell_df = pell_df.dropna()
    
    # Change opeid to super_opeid
    for i in range(pell_df.shape[0]):
        opeid = str(pell_df[id].iloc[i])
        if add_zeros:
            opeid = opeid + "00"
            pell_df[id].iloc[i] = opeid
        if opeid in super_opeid_lookup.keys():
            pell_df[id].iloc[i] = super_opeid_lookup[opeid]
    
    # Sum across super_opeid
    recip_series = pell_df.groupby(id)[vars[0]].sum()
    award_series = pell_df.groupby(id)[vars[1]].sum()


    # Create new dataframe
    pell_df = pd.merge(recip_series, award_series, on=id)
    pell_df = pell_df.rename(columns = { vars[0]:'recipients_'+year[-2:], vars[1]:'awards_'+year[-2:]})
    pell_df['super_opeid'] = pell_df.index.astype('str')
    pell_df['super_opeid'] = pell_df['super_opeid'].replace(r'\.0$','',regex=True)
    
    # Merge with df
    df = df.merge(pell_df, on='super_opeid', how='left')
    return df


df = add_pell_grants(df, '99-00', '.xls', ['Recipients', 'Awards'], 'OPEID', add_opeid=True)
df = add_pell_grants(df, '00-01', '.xls', ['Recipients', 'Awards'], 'OPEID', add_opeid=True)
df = add_pell_grants(df, '01-02', '.xls', ['Recipients', 'Awards'], 'OPEID', add_opeid=True)
df = add_pell_grants(df, '02-03', '.xls', ['Recipients', 'Awards'], 'OPEID')
df = add_pell_grants(df, '03-04', '.xls', ['Recipients', 'Awards'], 'OPEID')
df = add_pell_grants(df, '04-05', '.xls', ['Recipients', 'Awards'], 'OPEID')
df = add_pell_grants(df, '05-06', '.xls', ['Recipients', 'Awards'], 'OPEID')
df = add_pell_grants(df, '06-07', '.xls', ['Recipients', 'Awards'], 'OPEID')
df = add_pell_grants(df, '07-08', '.xls', ['Recips', 'Awards'], 'OPEID')
df = add_pell_grants(df, '08-09', '.xls', ['Recipients', 'Awards'], 'OPEID')
df = add_pell_grants(df, '09-10', '.xls', ['Recips', 'Awards'], 'OPEID', add_opeid=False, add_zeros=True)
df = add_pell_grants(df, '10-11', '.xls', ['Recipients', 'Awards'], 'OPEID')
df = add_pell_grants(df, '11-12', '.xls', ['Recips', 'Awards'], 'OPEID')
df = add_pell_grants(df, '12-13', '.xls', ['Total Recipients', 'Total Awards'], 'OPE ID')
df = add_pell_grants(df, '13-14', '.xls', [' Total Recipients ', ' Total Awards '], 'OPE_ID')
df = add_pell_grants(df, '14-15', '.xls', ['Total_Recipients', 'Total_Awards'], 'OPE_ID')
df = add_pell_grants(df, '15-16', '.xlsx', ['Total Recipients', 'Total Awards'], 'OPE ID')
df = add_pell_grants(df, '16-17', '.xlsx', [' TOTAL RECIPIENTS ', ' TOTAL AWARDS '], 'OPE_ID')
df = add_pell_grants(df, '17-18', '.xlsx', [' TOTAL RECIPIENTS ', ' TOTAL AWARDS '], 'OPE_ID')

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [8]:
# Fill in the missing pell grant with zeros. 
df = df.fillna(0)

In [9]:
df.head()

,super_opeid,par_median,k_median,par_q1,par_top1pc,kq5_cond_parq1,ktop1pc_cond_parq1,mr_kq5_pq1,mr_ktop1_pq1,trend_parq1,trend_bottom40,count,name,region,state,fips,cz,czname,cfips,county,zip,tier,tier_name,type,iclevel,public,barrons,exp_instr_pc_2000,exp_instr_pc_2013,multi,hbcu,flagship,ipeds_enrollment_2013,ipeds_enrollment_2000,sticker_price_2013,sticker_price_2000,grad_rate_150_p_2013,grad_rate_150_p_2002,avgfacsal_2013,avgfacsal_2001,...,pct_socialscience_2000,pct_tradepersonal_2000,recipients_00,awards_00,recipients_01,awards_01,recipients_02,awards_02,recipients_03,awards_03,recipients_04,awards_04,recipients_05,awards_05,recipients_06,awards_06,recipients_07,awards_07,recipients_08,awards_08,recipients_09,awards_09,recipients_10,awards_10,recipients_11,awards_11,recipients_12,awards_12,recipients_13,awards_13,recipients_14,awards_14,recipients_15,awards_15,recipients_16,awards_16,recipients_17,awards_17,recipients_18,awards_18
0,266500,30900.0,53000.0,36.477882,0.119815,44.843544,1.766630,16.357975,0.644429,-7.998776,-5.750611,207.666667,Vaughn College Of Aeronautics And Technology,1.0,NY,36.0,19400.0,New York,36081.0,Queens,11369.0,8.0,Nonselective four-year private not-for-profit,2.0,1.0,0.0,9.0,7395.8291,5508.3994,0.0,0.0,0.0,1724.0,1297.0,21642.0,7596.37459,0.555556,0.348837,9139.0,41498.637931,...,0.000000,43.506493,767.0,1565883.03,772.0,1682884.11,824.0,2132572.88,905.0,2391328.17,923.0,2415131.88,965.0,2248236.14,693.0,1830619.76,653.0,1744690.0,674.0,1968060.0,739.0,2327303.91,962.0,4094218.0,1182.0,5362281.0,1227.0,4989995.0,1329.0,5390374.0,1246.0,5164952.0,1236.0,5264086.0,1143.0,4803637.0,1025.0,4352283.0,993.0,4440556.0
1,727300,42800.0,57600.0,27.632242,0.559202,46.824234,2.556827,12.938586,0.706509,-9.186549,-12.297223,1083.000000,CUNY Bernard M. Baruch College,1.0,NY,36.0,19400.0,New York,36061.0,New York,10010.0,5.0,Selective public,1.0,1.0,1.0,3.0,3385.4727,6754.6748,0.0,0.0,0.0,14082.0,15698.0,6561.0,3350.00000,0.449033,0.447022,8651.0,48525.348032,...,9.132841,0.000000,0.0,0.00,0.0,0.00,0.0,0.00,4932.0,14883698.35,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,5209.0,16753486.0,0.0,0.00,5564.0,23756684.0,6391.0,28892165.0,7037.0,30224964.0,7014.0,30386258.0,6950.0,30445365.0,7623.0,33609555.0,7671.0,33728070.0,7565.0,33739177.0,0.0,0.0
2,268800,35500.0,48500.0,32.546474,0.233515,36.021557,1.408721,11.723747,0.458489,-9.801580,-13.879366,582.333333,City College Of New York - CUNY,1.0,NY,36.0,19400.0,New York,36061.0,New York,10031.0,5.0,Selective public,1.0,1.0,1.0,4.0,3095.1514,9341.7002,0.0,0.0,0.0,12501.0,11055.0,6389.0,3309.00000,0.476538,0.377361,8257.0,65619.116696,...,35.803658,0.000000,0.0,0.00,0.0,0.00,0.0,0.00,3967.0,11314481.67,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,5400.0,16984794.0,0.0,0.00,6534.0,27504175.0,6884.0,30842029.0,7302.0,31412452.0,7074.0,30790354.0,7014.0,30977262.0,7377.0,32777164.0,7359.0,33055193.0,7444.0,34072456.0,0.0,0.0
3,702200,32500.0,40700.0,36.707489,0.000000,27.882967,0.189635,10.235138,0.069610,-5.733966,-9.072347,468.333333,CUNY Lehman College,1.0,NY,36.0,19400.0,New York,36005.0,Bronx,10468.0,5.0,Selective public,1.0,1.0,1.0,5.0,3842.9580,7139.1030,0.0,0.0,0.0,9886.0,8768.0,6408.0,3320.00000,0.449033,0.447022,7271.0,48525.348032,...,36.814621,0.000000,0.0,0.00,0.0,0.00,0.0,0.00,4204.0,11700371.71,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,5130.0,15212050.0,0.0,0.00,5958.0,22975756.0,6241.0,26438892.0,6243.0,25183064.0,5849.0,23140618.0,6032.0,24507196.0,6492.0,26836607.0,6718.0,28224374.0,6890.0,29210253.0,0.0,0.0
4,114000,36600.0,43000.0,33.116928,0.155981,29.949804,0.083620,9.918455,0.027692,-13.313572,-14.919846,1179.666667,"California State University, Los Angeles",4.0,CA,6.0,38300.0,Los Angeles,6037.0,Los Angeles,90032.0,5.0,Selective public,1.0,1.0,1.0,4.0,3301.5615,4761.4780,0.0,0.0,0.0,19589.0,19593.0,6348.0,1724.00000,0.356537,0.318355,8767.0,68806.605852,...,29.879999,0.000000,6745.0,14797742.43,6777.0,15956798.13,7406.0,19408263.13,7502.0,20079040.27,7614.0,20451692.25,

In [11]:
# Export data
df.to_csv('cleaned_dataframe.csv', index=False)